In [1]:
import torchvision
import torch 
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.dataloader import DataLoader
from mydataset import MyDataset, acc
import random

import copy

nways = 4

In [2]:
bsize = 8
ksize = 5
dataset = MyDataset(k=ksize+1, n=nways, ntrain=800, is_train=True)
dataset_test = MyDataset(k=ksize, n=nways, ntrain=800, is_train=False)

loader = torch.utils.data.DataLoader(dataset, batch_size=bsize, num_workers=3)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=bsize, num_workers=3)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.pixs = 27**2
        self.lin1 = nn.Linear(512+self.pixs, 64)
        self.lin2 = nn.Linear(64, nways)
        self.lstm = nn.LSTM(self.pixs+nways, 512, num_layers=2, batch_first=True)

    def forward(self, x, idx, x2):
        x = x.view(x.shape[0], x.shape[1], x.shape[2], self.pixs).view(x.shape[0], -1, self.pixs)
        idx = idx.view(idx.shape[0], -1, nways).float()
        x2 = x2.view(x2.shape[0], nways, self.pixs)
        h = torch.cat((x, idx), dim=-1)
        _, (h, _) = self.lstm(h)
        h = h.view(2, 1, x.shape[0], -1)[1].squeeze(0).unsqueeze(1).repeat(1, nways, 1)
        h2 = torch.cat((x2, h), dim=-1)
        h2 = self.lin1(h2)
        h2 = F.relu(h2)
        h2 = self.lin2(h2)
        h2 = torch.softmax(h2, dim=-1)
        return h2
        
model = Model()

In [ ]:
class Model:
    def __init__(self, ):
        l = [nn.Conv2d(1, 32), nn.Conv2d(32, 64), nn.Conv2d(64, 64)]
        self.convs = nn.ModuleList(l)
        self.pool = nn.MaxPool2d(2)
        self.lin1 = nn.Linear(512, 64)
        self.lstm = nn.LSTM(64, 64, num_layers=1, batch_first=True)

    def forward(self, x_train, y_train, x_test, y_test):
        h = h0 = x_train.view(-1, 1, x.shape[-2], x.shape[-1])
        h = h0 = x_test.view(-1, 1, x.shape[-2], x.shape[-1])
        for conv in self.convs:
            h = conv(h)
            h = self.pool(h)
            h = F.relu(h)
        h = h.view(x_train.shape[0], -1, )
        h = F.relu(self.lin1(h))
        _, (h, _) = self.lstm(h)


In [4]:
opt = torch.optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
ename = 'tst_hdim512_b512_long'
os.system('mkdir tb/'+ename)
writer = SummaryWriter('tb/'+ename)

ii = 0
eps=1000
for ep in range(eps):
    for step, (x, y) in enumerate(loader):
        xexamples, xtoguess = x[:, :-1], x[:, -1]
        yexamples, ytoguess = y[:, :-1], y[:, -1]
        preds = model(xexamples, yexamples, xtoguess)
        loss = -(ytoguess*torch.log(preds+1e-8)).mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        accu = acc(ytoguess, preds)
        writer.add_scalar("loss/accuracy", accu, ii:=ii+1)
        writer.add_scalar("loss/loss", loss.item(), ii)
        if step%4==0:
            print(ep, step, loss.item(), accu)